# Polygon Extraction from Vector Layers

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/YOUR_REPO/blob/main/extract_polygons.ipynb)

This notebook extracts polygons from multiple vector layers (shapefiles) based on spatial coordinates and saves them to a GeoPackage file. The process includes:

1. Loading coordinates from a sampled data CSV file
2. Converting coordinates to spatial points
3. Extracting polygons that intersect with these points
4. Saving results to a multi-layer GeoPackage file

## Setup
First, let's install the required packages and import libraries.

In [ ]:
!pip install pandas geopandas shapely

In [ ]:
import pandas as pd
import geopandas as gpd
from pathlib import Path
import os
from shapely.geometry import Point
import warnings
warnings.filterwarnings('ignore')

## Define Helper Functions

### 1. Load Sampled Coordinates

In [ ]:
def load_sampled_coordinates(csv_path):
    """
    Load the coordinates from the sampled data CSV file
    Returns a DataFrame with unique coordinate pairs
    """
    df = pd.read_csv(csv_path)
    # Get unique coordinate pairs
    coords_df = df[['x_coord', 'y_coord']].drop_duplicates()
    return coords_df

### 2. Create Points GeoDataFrame

In [ ]:
def create_points_gdf(coords_df, crs):
    """
    Create a GeoDataFrame of points from coordinates
    """
    geometry = [Point(xy) for xy in zip(coords_df['x_coord'], coords_df['y_coord'])]
    return gpd.GeoDataFrame(coords_df, geometry=geometry, crs=crs)

### 3. Extract Polygons Function

In [ ]:
def extract_polygons(points_gdf, shapefile_path, buffer_distance=0):
    """
    Extract polygons that intersect with the points
    Optional buffer_distance parameter to create a buffer around points
    """
    # Read the shapefile
    polygons_gdf = gpd.read_file(shapefile_path)
    
    # Ensure both GeoDataFrames have the same CRS
    if points_gdf.crs != polygons_gdf.crs:
        points_gdf = points_gdf.to_crs(polygons_gdf.crs)
    
    # If buffer_distance is specified, create buffer around points
    if buffer_distance > 0:
        points_buffered = points_gdf.geometry.buffer(buffer_distance)
        # Spatial join between buffered points and polygons
        intersecting_polygons = gpd.sjoin(polygons_gdf, 
                                        gpd.GeoDataFrame(geometry=points_buffered, crs=points_gdf.crs),
                                        how='inner', 
                                        predicate='intersects')
    else:
        # Spatial join between points and polygons
        intersecting_polygons = gpd.sjoin(polygons_gdf, 
                                        points_gdf,
                                        how='inner', 
                                        predicate='contains')
    
    # Drop duplicate polygons and sjoin index
    intersecting_polygons = intersecting_polygons.drop_duplicates()
    if 'index_right' in intersecting_polygons.columns:
        intersecting_polygons = intersecting_polygons.drop(columns=['index_right'])
    
    return intersecting_polygons

## Main Processing

Now let's set up the main processing workflow. Make sure you have:
1. Your sampled coordinates CSV file in the working directory
2. A 'shapefiles' directory containing your vector layers

In [ ]:
# Define paths
sampled_data_path = "sampled_data_with_coords.csv"  # Path to your sampled coordinates CSV
shapefile_dir = "shapefiles"  # Directory containing your shapefiles
output_gpkg = "extracted_polygons.gpkg"  # Output GeoPackage file

# Check if input files/directories exist
if not os.path.exists(sampled_data_path):
    raise FileNotFoundError(f"Sampled data file not found: {sampled_data_path}")
if not os.path.exists(shapefile_dir):
    raise FileNotFoundError(f"Shapefile directory not found: {shapefile_dir}")

# Load coordinates
coords_df = load_sampled_coordinates(sampled_data_path)
print(f"Loaded {len(coords_df)} unique coordinate pairs")

# Create points GeoDataFrame (assuming WGS 84 - EPSG:4326, adjust if different)
points_gdf = create_points_gdf(coords_df, crs="EPSG:4326")

## Process Shapefiles and Save Results

In [ ]:
# Process each shapefile
for shapefile in Path(shapefile_dir).glob("*.shp"):
    layer_name = shapefile.stem
    print(f"Processing {layer_name}")
    
    try:
        # Extract intersecting polygons
        intersecting_polygons = extract_polygons(points_gdf, shapefile)
        
        # Save to GeoPackage
        intersecting_polygons.to_file(output_gpkg, 
                                    layer=layer_name, 
                                    driver="GPKG",
                                    mode='a' if os.path.exists(output_gpkg) else 'w')
        
        print(f"Saved {len(intersecting_polygons)} polygons for layer {layer_name}")
        
    except Exception as e:
        print(f"Error processing {layer_name}: {str(e)}")

print(f"\nAll layers have been processed and saved to {output_gpkg}")